In [10]:
import sys
#sys.path.append('..')

import datetime
import POP_BO.popbo.util as util
from POP_BO.popbo.popbo import POPBO
import POP_BO.popbo.Oracles as sigma
import numpy as np
import sys
import time
import os
import pickle

In [11]:
test_prob_name_list = ['Branin', 'CurrinExp', 'Beale',
        'Styblinski_tang', 'Bukin', 'Cross_in_tray', 'Eggholder',
        'Holder_table', 'Langerman', 'Levy13', 'Shubert', "matyas"]
test_prob_names_to_run = test_prob_name_list#[test_prob_name_list[10]]
# test_prob_name_list[3:] #'Branin' #'CurrinExp' #'Forrester'

In [12]:
def run_one_instance_popbo(problem_dim, num_run, config):
    popbo = POPBO(config)
    f = config['obj']

    x_prime = [np.random.rand()]
    x = popbo.query_new_point(x_prime=x_prime)

    eval_data_list = []
    for k in range(num_run):
        x_prime = x
        x = popbo.query_new_point(x_prime=x_prime)
        one = sigma.pref_oracle(f(x), f(x_prime))
        popbo.add_new_data(x_prime=x_prime, x=x, one=one)
        eval_data_list.append([x, f(x)])

    data_list = []
    for k in range(num_run):
        data_list.append(
            [
                [
                    f(popbo.x_list[k]),
                    f(popbo.x_prime_list[k]),
                    popbo.one_list[k]]
            ]
        )

    opt_val = config['opt_val']
    return eval_data_list, data_list, opt_val

In [13]:
def run_one_instance_popbo_best_x(problem_dim, num_run, config):
    popbo = POPBO(config)
    f = config['obj']

    bounds = config['bounds']

    if problem_dim == 1:
        x_prime = [
            np.random.rand() * (bounds[1] - bounds[0]) + bounds[0]
        ]
    elif problem_dim == 2:
        x_prime = [
            np.random.rand() * (bounds[0][1] - bounds[0][0]) + bounds[0][0],
            np.random.rand() * (bounds[1][1] - bounds[1][0]) + bounds[1][0]
        ]

    x = popbo.query_new_point(x_prime=x_prime)
    opt_val = config['opt_val']
    cumu_reg = (opt_val - f(x_prime)) + (opt_val - f(x))
    eval_data_list = []
    for k in range(num_run):
        x_prime = x

        x = popbo.query_new_point(x_prime=x_prime)
        proj_x = [0] * problem_dim
        if problem_dim == 1:
            proj_x = max(min(x, bounds[1]), bounds[0])
        else:
            for j in range(problem_dim):
                proj_x[j] = max(min(x[j], bounds[j][1]), bounds[j][0])

        x = proj_x
        one = sigma.pref_oracle(f(x), f(x_prime))
        popbo.add_new_data(x_prime=x_prime, x=x, one=one)
        eval_data_list.append([x, f(x)])
        cumu_reg += (opt_val-f(x))
        print(f'Step {k}, instantaneous regret: {opt_val-f(x)},'+
              f'cumu_reg: {cumu_reg}.\n')


    data_list = []
    for k in range(num_run):
        data_list.append(
            [
                [f(popbo.x_list[k]),
                 f(popbo.x_prime_list[k]),
                 popbo.one_list[k]]
            ]
        )

    opt_val = config['opt_val']

    best_x_list = popbo.estimated_best_x_list
    best_x_val_list = popbo.estimated_best_x_val_list
    return eval_data_list, data_list, opt_val, best_x_list, best_x_val_list

In [ ]:
for test_prob_name in test_prob_names_to_run:
    eval_data_list_list_popbo = []
    data_list_list_popbo = []
    best_x_list_list_popbo = []
    best_x_val_list_list_popbo = []
    last_best_x_regret_list_list_popbo = []

    opt_val_list = []

    eval_data_list_list_qEUBO = []
    data_list_list_qEUBO = []

    eval_data_list_list_sgp = []
    data_list_list_sgp = []

    eval_data_list_list_GJ = []
    data_list_list_GJ = []

    instance_num = 30
    num_run = 30
    problem_dim = 2

    for k in range(instance_num):
        print(f'{test_prob_name}: {k}-th instance.')
        config = util.get_config('test_func_'+test_prob_name,
                                problem_dim=problem_dim)

        eval_data_list_popbo, data_list_popbo, opt_val, best_x_list_popbo, \
            best_x_val_list_popbo = run_one_instance_popbo_best_x(
            problem_dim, num_run, config
        )

        eval_data_list_list_popbo.append(eval_data_list_popbo)
        data_list_list_popbo.append(data_list_popbo)

        best_x_list_list_popbo.append(best_x_list_popbo)
        best_x_val_list_list_popbo.append(best_x_val_list_popbo)
        last_best_x_regret_list_list_popbo.append(
            opt_val - best_x_val_list_popbo[-1]
        )
        opt_val_list.append(opt_val)
        last_best_regret_mean = np.mean(last_best_x_regret_list_list_popbo)
        last_best_regret_std = np.std(last_best_x_regret_list_list_popbo)
        print(
            f'{k} instances: {last_best_regret_mean}, {last_best_regret_std}'
        )

    now_time_str = datetime.datetime.now().strftime(
            "%H_%M_%S-%b_%d_%Y")


    with open(f'../result/test_func_popbo_{test_prob_name}_' +
            f'{now_time_str}.pkl', 'wb') as f:
        pickle.dump([
            eval_data_list_list_popbo, data_list_list_popbo, opt_val_list,
            best_x_list_list_popbo, best_x_val_list_list_popbo
            ], f)


Branin: 0-th instance.
  index  |  Mat52.lengthscale  |  constraints  |  priors
  [0]    |        10.00000000  |      +ve      |        
  [1]    |        10.00000000  |      +ve      |        
[ 9.28839156 14.68970864] with opt improvement 6.000000055013919, real 6.000000055017841
[1.57898003 6.52683829] with opt improvement 6.000000054988189, real 6.00000005498819
Step 1: norm 5.750969812735887, LL value -0.0024756954606259818,avg -0.0024756954606259818.
Step 0, instantaneous regret: 0.29609598054244934,cumu_reg: 4.484593936407536.

[-3.36440289  1.90294151] with opt improvement 6.000000054993452, real 6.000000054992741
Estimated best x: [1.5789800287276907, 6.52683828897576],  with regret 0.29609598054244934.
Step 2: norm 9.999999379752229, LL value -0.006300308359846518,avg -0.003150154179923259.
Step 1, instantaneous regret: 2.113973956643548,cumu_reg: 6.598567893051084.

[5.72889411 8.21677405] with opt improvement 6.000000054995674, real 6.000000054989082
Estimated best x: [1.57

AttributeError: 'function' object has no attribute 'pref_oracle'